In [1]:
import pandas as pd

import nltk
import spacy

import string
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline 
from sklearn.metrics import classification_report

In [2]:
df2021= pd.read_csv('data/zeus_download_doc_digitalizzati20210514.csv', sep="|", encoding='latin')
df2021 = df2021.map(lambda x: x.lower() if isinstance(x, str) else x)
df2021.loc[df2021['famiglia'] == 'https://www.epc.bnl.it/bnl/filedownload.ashx?i=1&codicefase=5246330&num_prat=559377', 'famiglia'] = 'altro'
df2021.loc[df2021['famiglia'] == 'documenti non classificati', 'famiglia'] = 'altro'
df2021 = df2021[~df2021['nome_epc'].isna()]

df2021['original'] = df2021['nome_epc']
# Keep only alphabetic characters and spaces
df2021['nome_epc'] = df2021['original'].apply(lambda x: ''.join([c for c in x if c.isalpha() or c.isspace()]))
df2021['nome_epc'] = df2021['nome_epc'].str.replace(r'\s{2,}', ' ', regex=True)

# Trim leading and trailing whitespaces
df2021 = df2021.map(lambda x: x.strip() if isinstance(x, str) else x)

In [3]:
df2021

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
0,23.0,fatture,pubblicitaï¿½,02560563.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_387336_1_
1,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto
2,532.0,fatture,altro,00730131.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_88299_1_
3,946.0,altro,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
4,1538.0,altro,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
...,...,...,...,...,...,...,...
1299523,1375653.0,comunicazioni,altro,03290239.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299524,1375653.0,comunicazioni,corrispondenza con debitori/garanti,03334551.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299525,1375739.0,passaggi di status e relativa documentazione,ricevute di ritorno,03315819.zip,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299526,1375739.0,passaggi di status e relativa documentazione,ricevute di ritorno della lettera di messa in ...,03576134.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___


In [4]:
altri = df2021[df2021['famiglia'] == 'altro']
noti = df2021[df2021['famiglia'] != 'altro']

In [5]:
altri

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
1,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto
3,946.0,altro,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
4,1538.0,altro,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
6,1599.0,altro,altro,00331905.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
7,1599.0,altro,altro,00839638.pdf,legfiimm,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__
...,...,...,...,...,...,...,...
1299459,1367400.0,altro,altro,03384753.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
1299489,1372272.0,altro,altro,03460145.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299498,1372779.0,altro,altro,03597572.pdf,ritiro copiae esecutivae del titolo,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ritiro copia/e esecutiva/e del titolo
1299502,1374833.0,altro,altro,03484174.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento


In [6]:
noti

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
0,23.0,fatture,pubblicitaï¿½,02560563.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_387336_1_
2,532.0,fatture,altro,00730131.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_88299_1_
5,1538.0,fatture,altro,00655325.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_93992_1_
11,1730.0,fatture,altro,01883626.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_289834_1_
13,2032.0,perizie,valutazioni immobiliari,00106401.jpg,legricog,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legricog_3114_5420_3
...,...,...,...,...,...,...,...
1299522,1375611.0,contratti e documentazione relativa,contratto apertura rapporto (set completo),03166860.pdf,checklist,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,checklist_1_4825716_
1299523,1375653.0,comunicazioni,altro,03290239.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299524,1375653.0,comunicazioni,corrispondenza con debitori/garanti,03334551.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299525,1375739.0,passaggi di status e relativa documentazione,ricevute di ritorno,03315819.zip,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___


In [45]:
len(altri['nome_epc'].drop_duplicates())

18133

In [7]:
noti #to train the model

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
0,23.0,fatture,pubblicitaï¿½,02560563.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_387336_1_
2,532.0,fatture,altro,00730131.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_88299_1_
5,1538.0,fatture,altro,00655325.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_93992_1_
11,1730.0,fatture,altro,01883626.pdf,cpnotule,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,cpnotule_289834_1_
13,2032.0,perizie,valutazioni immobiliari,00106401.jpg,legricog,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legricog_3114_5420_3
...,...,...,...,...,...,...,...
1299522,1375611.0,contratti e documentazione relativa,contratto apertura rapporto (set completo),03166860.pdf,checklist,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,checklist_1_4825716_
1299523,1375653.0,comunicazioni,altro,03290239.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299524,1375653.0,comunicazioni,corrispondenza con debitori/garanti,03334551.msg,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299525,1375739.0,passaggi di status e relativa documentazione,ricevute di ritorno,03315819.zip,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___


In [8]:
filtered_df = df2021[['famiglia', 'nome_epc', 'original']]

In [9]:
counts_dict = {}

for famiglia_value in filtered_df['famiglia'].unique():
    
    # Filter DataFrame for the current 'famiglia'
    filtered = filtered_df[filtered_df['famiglia'] == famiglia_value]
    
    # Count occurrences of each 'nome_epc' for the current 'famiglia'
    nome_epc_counts = filtered['nome_epc'].value_counts().to_dict()
    
    # Add counts to the dictionary
    counts_dict[famiglia_value] = nome_epc_counts

counts_df = pd.DataFrame(counts_dict)
counts_df

,fatture,altro,perizie,atti giudiziali,visure,certificazioni contabili,comunicazioni,imposte,pubblicita',passaggi di status e relativa documentazione,proposte,garanzie e documentazione relativa,contratti e documentazione relativa,closing,delibere
cpnotule,164480.0,3319.0,38.0,1307.0,4.0,2.0,343.0,288.0,161.0,NaN,13.0,3.0,1.0,1.0,NaN
legnotule,69412.0,1537.0,1.0,34.0,NaN,NaN,10.0,27.0,NaN,NaN,5.0,1.0,NaN,NaN,NaN
documento note sulla procedura,2744.0,61961.0,971.0,16328.0,306.0,1263.0,6869.0,886.0,621.0,12.0,1120.0,69.0,143.0,5.0,NaN
documentogenerico,664.0,43451.0,1964.0,5514.0,5896.0,14351.0,11324.0,66.0,280.0,22037.0,1241.0,4524.0,9869.0,7727.0,12.0
messaggio generico,139.0,1434.0,8.0,320.0,14.0,86.0,303.0,55.0,9.0,NaN,24.0,5.0,6.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ordinario ordinario fideiussioni variazioni condizioni,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
inserimento documento mutuo nota discrizione ipoteca abbey national bank surrogato,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
estratto notarile prestito e contratto estratto notarile c e con contratto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
atto compravendita rep notaio mercurio paolo dragonetti,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [10]:
counts_df = counts_df.drop(columns='altro')

max_family_for_row = counts_df.idxmax(axis=1)

# Create a new DataFrame with 'word' as index and 'famiglia' as the corresponding column
result_df = pd.DataFrame({'word': max_family_for_row.index, 'famiglia': max_family_for_row.values})

result_df

C:\Users\sophia.dellarciprete\AppData\Local\Temp\ipykernel_1616\2599007868.py:3: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  max_family_for_row = counts_df.idxmax(axis=1)


,word,famiglia
0,cpnotule,fatture
1,legnotule,fatture
2,documento note sulla procedura,atti giudiziali
3,documentogenerico,passaggi di status e relativa documentazione
4,messaggio generico,atti giudiziali
...,...,...
22801,ordinario ordinario fideiussioni variazioni co...,contratti e documentazione relativa
22802,inserimento documento mutuo nota discrizione i...,contratti e documentazione relativa
22803,estratto notarile prestito e contratto estratt...,contratti e documentazione relativa
22804,atto compravendita rep notaio mercurio paolo d...,contratti e documentazione relativa


In [11]:
# result_df[result_df['famiglia'].isna()] #16310 
# result_df[~result_df['famiglia'].isna()] #6496

In [12]:
result_df.rename(columns={'word': 'nome_epc'}, inplace=True)

In [13]:
result_df = altri.merge(result_df, on = 'nome_epc', how = 'left')
result_df = result_df.drop(columns='famiglia_x')
result_df.rename(columns={'famiglia_y': 'famiglia'}, inplace=True)

previsti = result_df[~result_df['famiglia'].isna()]
da_fare = result_df[result_df['famiglia'].isna()]

In [14]:
previsti

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
0,532.0,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,certificazioni contabili
1,946.0,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
2,1538.0,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,passaggi di status e relativa documentazione
3,1599.0,altro,00331905.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
4,1599.0,altro,00839638.pdf,legfiimm,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__,perizie
...,...,...,...,...,...,...,...
546368,1367400.0,altro,03384753.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
546369,1372272.0,altro,03460145.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,passaggi di status e relativa documentazione
546370,1372779.0,altro,03597572.pdf,ritiro copiae esecutivae del titolo,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ritiro copia/e esecutiva/e del titolo,contratti e documentazione relativa
546371,1374833.0,altro,03484174.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure


In [15]:
da_fare

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
9,2339.0,altro,01489189.pdf,lettera perentorio avv rossi,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,lettera perentorio avv. rossi,NaN
12,3359.0,altro,00939663.pdf,comfidi,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,comfidi 30/1/2012,NaN
18,4359.0,altro,00372495.htm,messaggio ad avvocato,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,messaggio ad avvocato,NaN
41,7690.0,altro,00827375.msg,quietanza firmata da efibanca,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,quietanza firmata da efibanca,NaN
47,8941.0,altro,00355634.pdf,relazione geometra,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,5 -relazione geometra 2,NaN
...,...,...,...,...,...,...,...
545969,1278842.0,altro,02925644.pdf,integrazione compenso per procedimento di ingi...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,integrazione compenso per procedimento di ingi...,NaN
546060,1067292.0,altro,02222428.pdf,il legale esterno trasmette intervento di altr...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno trasmette intervento di altr...,NaN
546079,1067919.0,altro,02078192.pdf,rapporto cp,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,rapporto - cp 000000000001018730,NaN
546082,1068014.0,altro,02398442.pdf,il legale esterno richiede notifica ai sensi d...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno richiede notifica ai sensi d...,NaN


when the nome_epc in df2021 exists in the previsions_done when the famiglia is altro, it should replace the family with the value in previsions_done
df2021 should be filtered before to only contain altro, since we're gonna use it for the model, and we don't want our predictions in the model



In [16]:
df_clean = noti[['famiglia', 'nome_epc', 'original']].drop_duplicates()
df_clean

,famiglia,nome_epc,original
0,fatture,cpnotule,cpnotule_387336_1_
2,fatture,cpnotule,cpnotule_88299_1_
5,fatture,cpnotule,cpnotule_93992_1_
11,fatture,cpnotule,cpnotule_289834_1_
13,perizie,legricog,legricog_3114_5420_3
...,...,...,...
1299490,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca cons...,ipoteca ( 01 00007962706 (valida...
1299492,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca cons...,ipoteca ( 01 00007717454 (valida...
1299494,garanzie e documentazione relativa,ipoteca valida non ancora escussa ipoteca conv...,ipoteca ( 01 43000272238 (valida...
1299514,passaggi di status e relativa documentazione,informazioni ndg sare salumificio reggiano spa...,informazioni ndg (01005 sa.re salumificio regg...


In [17]:
nltk.download('stopwords')
nlp = spacy.load('it_core_news_sm')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sophia.dellarciprete\AppData\Roaming\nltk_dat
[nltk_data]     a...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
def procesar_texto(cadena):
    sinpuntuacion = [c for c in cadena if c not in string.punctuation]
    sinpuntuacion =' '.join(sinpuntuacion)
    palabras = sinpuntuacion.split()
    cadena_limpia = [palabra for palabra in palabras if palabra not in stopwords.words("italian")]
    return cadena_limpia

transformador = CountVectorizer(analyzer=procesar_texto).fit(df_clean['nome_epc'])

In [19]:
x_train, x_test, y_train, y_test = train_test_split(df_clean["nome_epc"], df_clean["famiglia"],test_size=0.3)

pipeline = Pipeline([
('vectorizar',CountVectorizer(analyzer = procesar_texto)),
('transformar', TfidfTransformer()), 
('clasificar', RandomForestClassifier())
])

In [20]:
pipeline.fit(x_train, y_train)

Pipeline(steps=[('vectorizar',
                 CountVectorizer(analyzer=<function procesar_texto at 0x0000019F103AF060>)),
                ('transformar', TfidfTransformer()),
                ('clasificar', RandomForestClassifier())])

In [21]:
predicciones = pipeline.predict(x_test)

In [22]:
print(classification_report(y_test, predicciones)) 

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(res

                                              precision    recall  f1-score   support

                             atti giudiziali       0.59      0.56      0.58      1634
                    certificazioni contabili       0.98      0.97      0.98      4379
                                     closing       0.00      0.00      0.00         2
                               comunicazioni       0.93      0.87      0.90      4855
         contratti e documentazione relativa       0.97      0.77      0.86       855
                                    delibere       0.00      0.00      0.00         0
                                     fatture       0.99      1.00      0.99     59845
          garanzie e documentazione relativa       0.97      0.85      0.91       964
                                     imposte       0.03      0.01      0.01       162
passaggi di status e relativa documentazione       0.60      0.94      0.73       468
                                     perizie       1.

c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sophia.dellarciprete\.conda\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [24]:
final_dataframe = da_fare

In [25]:
final_dataframe.head(1)

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
9,2339.0,altro,01489189.pdf,lettera perentorio avv rossi,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,lettera perentorio avv. rossi,NaN


In [26]:
predictions = pipeline.predict(final_dataframe['nome_epc'])
#sicuramente final_dataframe adesso e 'da fare'

In [27]:
predictions

array(['comunicazioni', 'visure', 'comunicazioni', ..., 'atti giudiziali',
       'atti giudiziali', 'atti giudiziali'], dtype=object)

In [28]:
final_dataframe = final_dataframe.drop(columns=['famiglia'])
final_dataframe['famiglia'] = predictions
final_dataframe

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
9,2339.0,altro,01489189.pdf,lettera perentorio avv rossi,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,lettera perentorio avv. rossi,comunicazioni
12,3359.0,altro,00939663.pdf,comfidi,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,comfidi 30/1/2012,visure
18,4359.0,altro,00372495.htm,messaggio ad avvocato,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,messaggio ad avvocato,comunicazioni
41,7690.0,altro,00827375.msg,quietanza firmata da efibanca,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,quietanza firmata da efibanca,atti giudiziali
47,8941.0,altro,00355634.pdf,relazione geometra,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,5 -relazione geometra 2,atti giudiziali
...,...,...,...,...,...,...,...
545969,1278842.0,altro,02925644.pdf,integrazione compenso per procedimento di ingi...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,integrazione compenso per procedimento di ingi...,atti giudiziali
546060,1067292.0,altro,02222428.pdf,il legale esterno trasmette intervento di altr...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno trasmette intervento di altr...,atti giudiziali
546079,1067919.0,altro,02078192.pdf,rapporto cp,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,rapporto - cp 000000000001018730,atti giudiziali
546082,1068014.0,altro,02398442.pdf,il legale esterno richiede notifica ai sensi d...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno richiede notifica ai sensi d...,atti giudiziali


In [35]:
previsti

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
0,532.0,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,certificazioni contabili
1,946.0,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
2,1538.0,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,passaggi di status e relativa documentazione
3,1599.0,altro,00331905.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
4,1599.0,altro,00839638.pdf,legfiimm,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__,perizie
...,...,...,...,...,...,...,...
546368,1367400.0,altro,03384753.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
546369,1372272.0,altro,03460145.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,passaggi di status e relativa documentazione
546370,1372779.0,altro,03597572.pdf,ritiro copiae esecutivae del titolo,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ritiro copia/e esecutiva/e del titolo,contratti e documentazione relativa
546371,1374833.0,altro,03484174.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure


In [37]:
done = pd.concat([previsti, final_dataframe], ignore_index=True)

In [38]:
done

,num_prat,des_tipodoc,nomefile,nome_epc,url,original,famiglia
0,532.0,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto,certificazioni contabili
1,946.0,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
2,1538.0,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___,passaggi di status e relativa documentazione
3,1599.0,altro,00331905.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento,visure
4,1599.0,altro,00839638.pdf,legfiimm,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__,perizie
...,...,...,...,...,...,...,...
546368,1278842.0,altro,02925644.pdf,integrazione compenso per procedimento di ingi...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,integrazione compenso per procedimento di ingi...,atti giudiziali
546369,1067292.0,altro,02222428.pdf,il legale esterno trasmette intervento di altr...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno trasmette intervento di altr...,atti giudiziali
546370,1067919.0,altro,02078192.pdf,rapporto cp,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,rapporto - cp 000000000001018730,atti giudiziali
546371,1068014.0,altro,02398442.pdf,il legale esterno richiede notifica ai sensi d...,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,il legale esterno richiede notifica ai sensi d...,atti giudiziali


In [39]:
altri

,num_prat,famiglia,des_tipodoc,nomefile,nome_epc,url,original
1,532.0,altro,altro,00511949.pdf,inserimento acconto,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,inserimento acconto
3,946.0,altro,altro,00946317.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
4,1538.0,altro,altro,02080862.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
6,1599.0,altro,altro,00331905.pdf,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
7,1599.0,altro,altro,00839638.pdf,legfiimm,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,legfiimm_2890__
...,...,...,...,...,...,...,...
1299459,1367400.0,altro,altro,03384753.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento
1299489,1372272.0,altro,altro,03460145.pdf,documentogenerico,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,documentogenerico___
1299498,1372779.0,altro,altro,03597572.pdf,ritiro copiae esecutivae del titolo,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,ritiro copia/e esecutiva/e del titolo
1299502,1374833.0,altro,altro,03484174.zip,allega documento,https://www.epc.bnl.it/bnl/filedownload.ashx?i...,allega documento


In [ ]:
#drop nome_epc
done = done.drop(columns=['nome_epc'])
#change original name to nome_epc
done.rename(columns={'original': 'nome_epc'}, inplace=True)

In [40]:
done.to_csv('data/zeus_download_doc_digitalizzati20210514_exALTRO.csv', index=False)